In [1]:
import os
import time
import copy
import argparse
import numpy as np
import matplotlib.pyplot as plt
import torch
from fvcore.nn import FlopCountAnalysis
import tqdm
from torch.utils.data import Dataset
from torchvision import transforms
import torch.nn as nn
from torchvision.utils import save_image
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug


In [3]:
def train(model, train_loader, test_loader, optimizer, scheduler, criterion, epochs):
    model = model.to(device)
    criterion = criterion.to(device)
    model.train()

    for epoch in range(epochs):
        print('Start Epoch #{}'.format(epoch+1))
        with tqdm.tqdm(total=len(train_loader)) as pbar:
            loss_avg, acc_avg, num_exp = 0, 0, 0
            for i_batch, datum in enumerate(train_loader):
                img = datum[0].float().to(device)
                if epoch == 0 and i_batch == 0:
                    flops = FlopCountAnalysis(model, img)
                    print('The FLOPs is {}'.format(flops.total()))
        #         if aug:
        #             if args.dsa:
        #                 img = DiffAugment(img, args.dsa_strategy, param=args.dsa_param)
        #             else:
        #                 img = augment(img, args.dc_aug_param, device=args.device)
                lab = datum[1].long().to(device)
                n_b = lab.shape[0]
                output = model(img)
                loss = criterion(output, lab)
                acc = np.sum(np.equal(np.argmax(output.cpu().data.numpy(), axis=-1), lab.cpu().data.numpy()))
                loss_avg += loss.item()*n_b
                acc_avg += acc
                num_exp += n_b

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                pbar.update(1)
                del img, lab
                
        loss_avg /= num_exp
        acc_avg /= num_exp
        scheduler.step()
        print('Train accuracy is {}%'.format(acc_avg*100))
        print('Average train loss is {}'.format(loss_avg))
        
    loss_avg, acc_avg, num_exp = 0, 0, 0   
    model.eval()
    for i_batch, datum in enumerate(test_loader):
        img = datum[0].float().to(device)
        lab = datum[1].long().to(device)
        n_b = lab.shape[0]
        output = model(img)
        loss = criterion(output, lab)
        acc = np.sum(np.equal(np.argmax(output.cpu().data.numpy(), axis=-1), lab.cpu().data.numpy()))
        loss_avg += loss.item()*n_b
        acc_avg += acc
        num_exp += n_b
        
        del img, lab

    loss_avg /= num_exp
    acc_avg /= num_exp
    print('Test accuracy is {}%'.format(acc_avg*100))
    print('Average test loss is {}'.format(loss_avg))          

    return loss_avg, acc_avg

class CustomDataset(Dataset):
    def __init__(self, images, labels, transform): # images: n x c x h x w tensor
        self.images = images.detach().float()
        self.labels = labels.detach()
        self.transform = transform

    def __getitem__(self, index):
        return self.transform(self.images[index]), self.labels[index]

    def __len__(self):
        return self.images.shape[0]
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data_path = './MNISTresult/realres_DC_MNIST_ConvNet_10ipc.pt'
data = torch.load(data_path)
images = data['data'][0][0]

# for img in images:
#     img = img*255
#     plt.imshow(img.permute(1,2,0), cmap='gray', vmin=0, vmax=255)
#     plt.show()
MNIST_dataset = 'MNIST'
MNIST_data_path = './MNISTdata'
MNIST_channel, MNIST_im_size, MNIST_num_classes, MNIST_class_names, MNIST_mean, MNIST_std, MNIST_dst_train, MNIST_dst_test, MNIST_testloader = get_dataset(MNIST_dataset, MNIST_data_path)
condensed_labs_train = torch.ones(10*MNIST_num_classes)
mean = [0.1307]
std = [0.3081]
transform = transforms.Compose([transforms.Normalize(mean=mean, std=std)])

for c in range(MNIST_num_classes):  
    condensed_labs_train[c*10:(c+1)*10]*=c
    
condensed_train_dst = CustomDataset(images, condensed_labs_train, transform)
MNIST_trainloader = torch.utils.data.DataLoader(condensed_train_dst, batch_size=8, shuffle=True, num_workers=0)
model = get_network('AlexNet', MNIST_channel, MNIST_num_classes, MNIST_im_size).to(device) # get a random model
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0)
epochs = 20
start = time.time()
train(model, MNIST_trainloader , MNIST_testloader , optimizer, scheduler, criterion, epochs)
print('Training on the original MNIST dataset takes {} seconds'.format(time.time()-start))




Start Epoch #1


 31%|█████████████████████████▌                                                         | 4/13 [00:00<00:00, 33.56it/s]

The FLOPs is 1907605504


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 46.42it/s]


Train accuracy is 8.0%
Average train loss is 2.315928840637207
Start Epoch #2


...

100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 192.84it/s]


Train accuracy is 84.0%
Average train loss is 0.9929377126693726
Start Epoch #20


100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 187.66it/s]

Train accuracy is 89.0%
Average train loss is 0.9691627073287964


Test accuracy is 83.32000000000001%
Average test loss is 1.6897642930984498
Training on the original MNIST dataset takes 3.576970100402832 seconds


In [2]:
def train(model, train_loader, test_loader, optimizer, scheduler, criterion, epochs):
    model = model.to(device)
    criterion = criterion.to(device)
    model.train()

    for epoch in range(epochs):
        print('Start Epoch #{}'.format(epoch+1))
        with tqdm.tqdm(total=len(train_loader)) as pbar:
            loss_avg, acc_avg, num_exp = 0, 0, 0
            for i_batch, datum in enumerate(train_loader):
                img = datum[0].float().to(device)
                if epoch == 0 and i_batch == 0:
                    flops = FlopCountAnalysis(model, img)
                    print('The FLOPs is {}'.format(flops.total()))
        #         if aug:
        #             if args.dsa:
        #                 img = DiffAugment(img, args.dsa_strategy, param=args.dsa_param)
        #             else:
        #                 img = augment(img, args.dc_aug_param, device=args.device)
                lab = datum[1].long().to(device)
                n_b = lab.shape[0]
                output = model(img)
                loss = criterion(output, lab)
                acc = np.sum(np.equal(np.argmax(output.cpu().data.numpy(), axis=-1), lab.cpu().data.numpy()))
                loss_avg += loss.item()*n_b
                acc_avg += acc
                num_exp += n_b

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                pbar.update(1)
                del img, lab
                
        loss_avg /= num_exp
        acc_avg /= num_exp
        scheduler.step()
        print('Train accuracy is {}%'.format(acc_avg*100))
        print('Average train loss is {}'.format(loss_avg))
        
    loss_avg, acc_avg, num_exp = 0, 0, 0   
    model.eval()
    for i_batch, datum in enumerate(test_loader):
        img = datum[0].float().to(device)
        lab = datum[1].long().to(device)
        n_b = lab.shape[0]
        output = model(img)
        loss = criterion(output, lab)
        acc = np.sum(np.equal(np.argmax(output.cpu().data.numpy(), axis=-1), lab.cpu().data.numpy()))
        loss_avg += loss.item()*n_b
        acc_avg += acc
        num_exp += n_b
        
        del img, lab

    loss_avg /= num_exp
    acc_avg /= num_exp
    print('Test accuracy is {}%'.format(acc_avg*100))
    print('Average test loss is {}'.format(loss_avg))          

    return loss_avg, acc_avg

class CustomDataset(Dataset):
    def __init__(self, images, labels, transform): # images: n x c x h x w tensor
        self.images = images.detach().float()
        self.labels = labels.detach()
        self.transform = transform

    def __getitem__(self, index):
        return self.transform(self.images[index]), self.labels[index]

    def __len__(self):
        return self.images.shape[0]
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data_path = './MNISTresult/noiseres_DC_MNIST_ConvNet_10ipc.pt'
data = torch.load(data_path)
images = data['data'][0][0]

# for img in images:
#     img = img*255
#     plt.imshow(img.permute(1,2,0), cmap='gray', vmin=0, vmax=255)
#     plt.show()
MNIST_dataset = 'MNIST'
MNIST_data_path = './MNISTdata'
MNIST_channel, MNIST_im_size, MNIST_num_classes, MNIST_class_names, MNIST_mean, MNIST_std, MNIST_dst_train, MNIST_dst_test, MNIST_testloader = get_dataset(MNIST_dataset, MNIST_data_path)
condensed_labs_train = torch.ones(10*MNIST_num_classes)
mean = [0.1307]
std = [0.3081]
transform = transforms.Compose([transforms.Normalize(mean=mean, std=std)])

for c in range(MNIST_num_classes):  
    condensed_labs_train[c*10:(c+1)*10]*=c
    
condensed_train_dst = CustomDataset(images, condensed_labs_train, transform)
MNIST_trainloader = torch.utils.data.DataLoader(condensed_train_dst, batch_size=8, shuffle=True, num_workers=0)
model = get_network('AlexNet', MNIST_channel, MNIST_num_classes, MNIST_im_size).to(device) # get a random model
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0)
epochs = 20
start = time.time()
train(model, MNIST_trainloader , MNIST_testloader , optimizer, scheduler, criterion, epochs)
print('Training on the original MNIST dataset takes {} seconds'.format(time.time()-start))




Start Epoch #1


  8%|██████▍                                                                            | 1/13 [00:02<00:34,  2.88s/it]

The FLOPs is 1907605504


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:02<00:00,  4.41it/s]


Train accuracy is 8.0%
Average train loss is 2.328088493347168
Start Epoch #2


...

100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 167.34it/s]


Train accuracy is 95.0%
Average train loss is 0.4528187727928162
Start Epoch #20


100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 215.12it/s]

Train accuracy is 97.0%
Average train loss is 0.4353028154373169


Test accuracy is 84.38%
Average test loss is 1.3267022100448609
Training on the original MNIST dataset takes 6.3093321323394775 seconds
